## 그래프(Graph)

In [9]:
import tensorflow as tf

# 1x2 행렬을 만드는 constant op(작업)를 만들자
# 이 op는 default graph에 노드로 들어갈 것입니다.
# 생성함수에서 나온 값은 constant op의 결과값 입니다.

matrix1 = tf.constant([[3., 3.]])

# 2x1 행렬을 만드는 constant op를 만들자

matrix2 = tf.constant([[2.],[2.]])

# matrix1과 matrix2를 입력값으로 하는 Matmul op을 만들어 봅시다. (행렬 곱)
# 이 op의 결과값인 product는 행렬곱의 결과를 의미

product = tf.matmul(matrix1, matrix2)


################################################################################

# session에서 graph 실행하기

# default graph를 실행시켜 봅시다.
sess = tf.Session()

# 행렬 곱 작업을 실행하기 위해 session의 run() 메소드 호출
# 작업 결과값인 product 값을 넘겨줍니다.

# 작업에 필요한 모든 입력값들은 자동적으로 session에서 실행되며 보통은 병렬로 처리됩니다.
# run(product)가 실행되면 2개의 상수와 1개의 행렬곱으로 3개의 op가 실행됩니다.
# 작업의 결과물은 numpy 'ndarray' 오브젝트인 result 값으로 나옵니다.

result = sess.run(product)
print(result)
# 예상 결과 값 = [[12.]]

# 실행을 마치면 Session을 닫습니다.
sess.close()

[[12.]]


## session을 쉽게 관리하는 방법

In [10]:
import tensorflow as tf

matrix1 = tf.constant([[3., 3.]])
matrix2 = tf.constant([[2.],[2.]])

product = tf.matmul(matrix1, matrix2)

# 쉽게 관리하기 위해 with 구문 사용
# 자동으로 close
with tf.Session() as sess:
    result = sess.run([product])
    print(result)

[array([[12.]], dtype=float32)]


## CPU나 GPU 지정

In [12]:
import tensorflow as tf

with tf.Session() as sess:
    # device 구문을 사용
    with tf.device("/cpu:0"):
        # cpu : /cpu:0
        # gpu : /gpu:0
        # gpu_num_2 : /gpu:1
        matrix1 = tf.constant([[3., 3.]])
        matrix2 = tf.constant([[2.],[2.]])
        product = tf.matmul(matrix1, matrix2)
        
        result = sess.run([product])
        print(result)

[array([[12.]], dtype=float32)]


## 인터렉티브(interative)한 이용법

In [13]:
import tensorflow as tf
sess = tf.InteractiveSession()

x = tf.Variable([1.0, 2.0])
a = tf.constant([3.0, 3.0])

# 초기화 op의 run() 메서드를 이용해서 x를 초기화
x.initializer.run()

# x에서 a를 빼는 작업을 추가하고 실행시켜서 결과를 봅시다.
sub = tf.subtract(x, a)
print(sub.eval())
# eval() 메소드로 실행시킬 수 있음

sess.close()

[-2. -1.]


 ## 변수(Variables)

In [16]:
import tensorflow as tf

# 값이 0인 스칼라로 초기화된 변수를 만듭시다.
state = tf.Variable(0, name="counter")

# state에 1을 더하는 작업을 만듭니다.
one = tf.constant(1)
new_value = tf.add(state, one)
update = tf.assign(state, new_value)
# assign은 대입하는 메소드

# 그래프를 한 번 작동시킨 후에는 init 작업을 실행해서 변수를 초기화해야 합니다. 
init_op = tf.global_variables_initializer()

# graph와 작업들을 실행
with tf.Session() as sess:
    # init 작업 실행
    sess.run(init_op)
    # state의 시작값 출력
    print(sess.run(state))
    # state값을 업데이트하고 출력하는 작업을 실행
    for _ in range(3):
        sess.run(update)
        print(sess.run(state))

0
1
2
3


## Fetches

In [18]:
# 여러개 tesor를 받아오기
import tensorflow as tf

input1 = tf.constant([3, 0])
input2 = tf.constant([2, 0])
input3 = tf.constant([5, 0])
intermed = tf.add(input2, input3)
mul = tf.multiply(input1, intermed)

with tf.Session() as sess:
    # mul과 intermed를 각각 수행해서 출력
    result = sess.run([mul, intermed])
    print(result)
    

[array([21,  0]), array([7, 0])]


## Feeds

In [20]:
# graph의 연산에게 직접 tensor 값을 줄 수 있는 feed 메커니즘
# tf.placeholder를 사용하여 특정 작업을 feed 작업으로 지정

import tensorflow as tf

input1 = tf.placeholder(tf.float32)
input2 = tf.placeholder(tf.float32)

output = input1 * input2

with tf.Session() as sess:
    print(sess.run([output], feed_dict = {input1:[7.], input2:[2.]}))

[array([14.], dtype=float32)]


------------------------------------------

# MNIST

In [23]:
# MNIST 데이터셋 다운로드하는 파이썬 코드
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


## 회귀 구현하기(소프트맥스)

<img src="https://tensorflowkorea.gitbooks.io/tensorflow-kr/content/g3doc/images/softmax-regression-vectorequation.png" alt="SoftMax" style="width: 600px; height: auto;"/> 

In [25]:
import tensorflow as tf

# 784 차원의 벡터로 변형된 MNIST 이미지의 데이터를 넣으려고 합니다.
# None은 해당차원 길이가 어떤 길이든지 될 수 있음을 의미
x = tf.placeholder(tf.float32, [None, 784])

# 초기값
w = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))
# w가 [784, 10]의 형태를 갖는 이유는 w에 784차원의 이미지 벡터를 곱해서 각 클래스에 대한 증거값을 나타내는 10차원 벡터를 얻고자 하기 때문
# b는 10차원 벡터에 더하기 위해 [10]의 형태를 갖는 것입니다.

# 모델 구현
y = tf.nn.softmax(tf.matmul(x, w) + b)

# matmul로 x와 w를 먼저 곱하는데 x가 먼저인 경우는 행렬 곱 순서에 따른 수행시간때문에 앞에 둔것(수행시간 고려 매우 중요!)
# softmax 메소드는 tf.nn.softmax

## 학습

### 크로스 엔트로피

In [28]:
# 크로스 엔트로피로 효율성을 검사하는 것
y_ = tf.placeholder(tf.float32, [None, 10])

cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))

# 우선 tf.log로 y의 각 원소의 로그 값 계산
# y_의 각 원소를 tf.log(y)의 해당되는 원소들과 곱
# tf.reduce_sum으로 y의 2번째 차원(reduction_indices[1] 이라는 파라미터가 주어졌으므로)의 원소들을 합합니다.
# 마지막으로 tf.reduce_mean으로 배치(batch)의 모든 예시에 대한 평균을 계산합니다.
# 단, 수학적으로 불안정한 계산
# 정규화되지 않은 로짓(logit)에 대해 tf.nn.softmax_cross-entropy_with_logits을 적용

In [29]:
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)
# 여기서는 텐서플로우에게 학습 비율 0.5로 경사 하강법을 적용하여 크로스 엔트로피를 최소화하도록 지시
# 경사하강법이란, 텐서플로우가 각각의 변수를 비용을 줄이는 방향으로 조금씩 이동시키는 매우 단순한 방법(최적화 알고리즘)

In [30]:
init = tf.global_variables_initializer()

sess = tf.Session()
sess.run(init)

# 학습을 1000번
for i in range(1000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})

## 모델 평가하기

In [32]:
# 우선 모델이 라벨을 올바르게 예측했는지 확인
# tf.argmax는 텐서 안에서 특정 축을 따라 가장 큰 값의 인덱스를 찾기에 매우 유용한 함수
# 예로 tf.argmax(y, 1)는 우리의 모델이 생각하기에 각 데이터에 가장 적합하다고 판단한(가장 증거값이 큰) 라벨
# tf.argmax(y_, 1)는 실제 라벨
# tf.equal을 사용하여 예측이 맞았는지 확인

correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
# 이렇게하면 부울 값으로 이루어진 리스트를 얻게 됩니다.
# 이 값을 부동 소수점으로 바꿀 수 있습니다.

accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

0.9191
